<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Detecting_dementia_from_speech_and_transcripts_using_transformers_May243.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import tarfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
from transformers import (BertTokenizer, BertModel, ViTModel, ViTFeatureExtractor,
                         Wav2Vec2ForCTC, Wav2Vec2Processor, WhisperProcessor,
                         WhisperForConditionalGeneration)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import json
import glob
import re
from pathlib import Path
import soundfile as sf
from collections import defaultdict
from scipy import ndimage
import pickle
warnings.filterwarnings('ignore')

class SpeechTranscriber:
    """Automatic Speech Recognition for generating transcripts from audio"""

    def __init__(self, model_name="openai/whisper-base", cache_dir="./asr_cache"):
        """
        Initialize ASR model
        Options:
        - openai/whisper-base: Good balance of speed/accuracy
        - facebook/wav2vec2-base-960h: Faster but less accurate
        - openai/whisper-small: More accurate but slower
        """
        self.model_name = model_name
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

        print(f"Loading ASR model: {model_name}")

        if "whisper" in model_name.lower():
            self.processor = WhisperProcessor.from_pretrained(model_name)
            self.model = WhisperForConditionalGeneration.from_pretrained(model_name)
            self.asr_type = "whisper"
        else:
            self.processor = Wav2Vec2Processor.from_pretrained(model_name)
            self.model = Wav2Vec2ForCTC.from_pretrained(model_name)
            self.asr_type = "wav2vec2"

        # Move to GPU if available
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)
        self.model.eval()

        print(f"ASR model loaded on {self.device}")

    def transcribe_audio_file(self, audio_path, use_cache=True):
        """Transcribe a single audio file"""
        audio_path = Path(audio_path)

        # Check cache first
        cache_file = self.cache_dir / f"{audio_path.stem}_transcript.pkl"
        if use_cache and cache_file.exists():
            try:
                with open(cache_file, 'rb') as f:
                    cached_result = pickle.load(f)
                return cached_result['transcript']
            except:
                pass  # Cache corrupted, proceed with transcription

        try:
            # Load audio
            audio, sr = librosa.load(str(audio_path), sr=16000)  # Whisper expects 16kHz

            # Handle empty or very short audio
            if len(audio) < 1600:  # Less than 0.1 seconds
                transcript = ""
            else:
                transcript = self._transcribe_audio_array(audio)

            # Cache result
            if use_cache:
                try:
                    with open(cache_file, 'wb') as f:
                        pickle.dump({
                            'audio_path': str(audio_path),
                            'transcript': transcript,
                            'model': self.model_name
                        }, f)
                except:
                    pass  # Caching failed, but transcription succeeded

            return transcript

        except Exception as e:
            print(f"Error transcribing {audio_path}: {e}")
            return f"[Transcription failed for {audio_path.name}]"

    def _transcribe_audio_array(self, audio_array):
        """Transcribe audio array using the loaded model"""
        try:
            if self.asr_type == "whisper":
                return self._whisper_transcribe(audio_array)
            else:
                return self._wav2vec2_transcribe(audio_array)
        except Exception as e:
            print(f"Transcription error: {e}")
            return "[Transcription error]"

    def _whisper_transcribe(self, audio_array):
        """Transcribe using Whisper model"""
        # Process audio
        inputs = self.processor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features.to(self.device)

        # Generate transcription
        with torch.no_grad():
            predicted_ids = self.model.generate(inputs, max_length=448)

        # Decode
        transcript = self.processor.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcript.strip()

    def _wav2vec2_transcribe(self, audio_array):
        """Transcribe using Wav2Vec2 model"""
        # Process audio
        inputs = self.processor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        ).input_values.to(self.device)

        # Get logits
        with torch.no_grad():
            logits = self.model(inputs).logits

        # Decode
        predicted_ids = torch.argmax(logits, dim=-1)
        transcript = self.processor.batch_decode(predicted_ids)[0]

        return transcript.strip().lower()

    def batch_transcribe(self, audio_paths, batch_size=8):
        """Transcribe multiple audio files with progress bar"""
        transcripts = {}

        print(f"Transcribing {len(audio_paths)} audio files...")

        for audio_path in tqdm(audio_paths, desc="Transcribing"):
            transcript = self.transcribe_audio_file(audio_path)
            participant_id = self._extract_participant_id(Path(audio_path).name)
            transcripts[participant_id] = transcript

        return transcripts

    def _extract_participant_id(self, filename):
        """Extract participant ID from filename"""
        patterns = [
            r'adrso?(\d{3})',         # adrs0123 or adrso123
            r'adrsp?(\d{3})',         # adrsp123
            r'adrspt?(\d{1,3})',      # adrspt1, adrspt12
            r'(\d{3})',               # 3-digit numbers
            r'([A-Z]\d{2,3})',        # Letter followed by 2-3 digits
            r'(S\d{3})',              # S followed by 3 digits
        ]

        for pattern in patterns:
            match = re.search(pattern, filename)
            if match:
                return match.group(1) if pattern.startswith(r'(\d') else match.group(0)

        return Path(filename).stem

In [12]:
class EnhancedADReSSDataProcessor:
    """Enhanced ADReSS data processor with automatic speech recognition"""

    def __init__(self, output_dir='./extracted_data', asr_model="openai/whisper-base"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Initialize ASR
        self.transcriber = SpeechTranscriber(model_name=asr_model)

    def extract_adress_dataset(self, tar_path, dataset_name):
        """Extract ADReSS dataset and organize files properly"""
        extract_path = self.output_dir / dataset_name
        extract_path.mkdir(exist_ok=True)

        print(f"Extracting {tar_path} to {extract_path}")

        try:
            with tarfile.open(tar_path, 'r:gz') as tar:
                tar.extractall(path=extract_path)
            print(f"Successfully extracted {dataset_name}")

            # Find the actual dataset directory structure
            self._explore_directory_structure(extract_path)
            return extract_path
        except Exception as e:
            print(f"Error extracting {tar_path}: {e}")
            return None

    def _explore_directory_structure(self, base_path):
        """Explore and print directory structure"""
        print(f"\nDirectory structure for {base_path.name}:")
        for root, dirs, files in os.walk(base_path):
            level = root.replace(str(base_path), '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... and {len(files) - 5} more files")

    def process_adress_dataset_with_asr(self, extract_path):
        """Process ADReSS dataset with automatic speech recognition"""
        dataset_info = {
            'audio_files': [],
            'transcript_files': [],
            'metadata_files': [],
            'labels': {},
            'paired_data': [],
            'generated_transcripts': {}
        }

        # Look for ADReSS structure
        adress_dirs = list(extract_path.rglob("*ADReSS*"))
        if adress_dirs:
            main_dir = adress_dirs[0]
        else:
            main_dir = extract_path

        print(f"Processing from directory: {main_dir}")

        # Find audio files
        audio_patterns = ['**/*.wav', '**/*.mp3', '**/*.flac']
        for pattern in audio_patterns:
            dataset_info['audio_files'].extend(list(main_dir.glob(pattern)))

        print(f"Found {len(dataset_info['audio_files'])} audio files")

        # Generate transcripts using ASR
        print("Generating transcripts using ASR...")
        audio_paths = [str(path) for path in dataset_info['audio_files']]
        generated_transcripts = self.transcriber.batch_transcribe(audio_paths)
        dataset_info['generated_transcripts'] = generated_transcripts

        print(f"Generated {len(generated_transcripts)} transcripts")

        # Process labels from directory structure
        labels = self._extract_labels_from_structure(dataset_info['audio_files'])
        dataset_info['labels'] = labels

        # Create paired dataset with generated transcripts
        paired_data = self._create_paired_data_with_asr(dataset_info)
        dataset_info['paired_data'] = paired_data

        return dataset_info

    def _extract_labels_from_structure(self, audio_files):
        """Extract labels from file paths or directory structure"""
        labels = {}

        for audio_file in audio_files:
            # Extract participant ID
            participant_id = self._extract_participant_id(audio_file.name)

            # Determine label from path
            path_str = str(audio_file).lower()
            if '/ad/' in path_str or 'dementia' in path_str or 'alzheimer' in path_str:
                label = 1  # AD/Dementia
                class_name = 'AD'
            elif '/cn/' in path_str or 'control' in path_str or 'normal' in path_str:
                label = 0  # Control/Normal
                class_name = 'CN'
            elif 'decline' in path_str:
                label = 1  # Decline/progression
                class_name = 'AD'
            elif 'no_decline' in path_str or 'no-decline' in path_str:
                label = 0  # No decline
                class_name = 'CN'
            else:
                # Default classification based on filename patterns
                if any(marker in audio_file.name.lower() for marker in ['ad', 'dem', 'alz']):
                    label = 1
                    class_name = 'AD'
                else:
                    label = 0  # Default to control
                    class_name = 'CN'

            labels[participant_id] = {
                'label': label,
                'class_name': class_name,
                'audio_path': audio_file
            }

        return labels

    def _extract_participant_id(self, filename):
        """Extract participant ID from filename"""
        patterns = [
            r'adrso?(\d{3})',
            r'adrsp?(\d{3})',
            r'adrspt?(\d{1,3})',
            r'(\d{3})',
            r'([A-Z]\d{2,3})',
            r'(S\d{3})',
        ]

        for pattern in patterns:
            match = re.search(pattern, filename)
            if match:
                return match.group(1) if pattern.startswith(r'(\d') else match.group(0)

        return Path(filename).stem

    def _create_paired_data_with_asr(self, dataset_info):
        """Create paired audio-transcript dataset using ASR-generated transcripts"""
        paired_data = []

        # Create paired dataset using generated transcripts
        for participant_id, label_info in dataset_info['labels'].items():
            # Get generated transcript
            transcript = dataset_info['generated_transcripts'].get(participant_id, "")

            # Clean and validate transcript
            transcript = self._clean_and_validate_transcript(transcript)

            # If transcript is still empty or invalid, create a meaningful placeholder
            if not transcript or len(transcript.strip()) < 10:
                transcript = f"Audio sample from participant {participant_id}. Speech content unclear or silent."

            paired_data.append({
                'participant_id': participant_id,
                'audio_path': str(label_info['audio_path']),
                'transcript': transcript,
                'label': label_info['label'],
                'class_name': label_info['class_name'],
                'transcript_source': 'ASR_generated'
            })

        print(f"Created {len(paired_data)} paired samples with ASR transcripts")

        # Print class distribution
        labels = [item['label'] for item in paired_data]
        unique, counts = np.unique(labels, return_counts=True)
        for cls, count in zip(unique, counts):
            class_name = 'CN' if cls == 0 else 'AD'
            print(f"  {class_name}: {count} samples ({count/len(labels)*100:.1f}%)")

        # Print sample transcripts for verification
        print("\nSample generated transcripts:")
        print("-" * 50)
        for i, sample in enumerate(paired_data[:3]):
            print(f"Participant {sample['participant_id']} ({sample['class_name']}):")
            print(f"Transcript: {sample['transcript'][:100]}...")
            print()

        return paired_data

    def _clean_and_validate_transcript(self, transcript):
        """Clean and validate ASR-generated transcript"""
        if not transcript:
            return ""

        # Remove common ASR artifacts
        transcript = transcript.strip()
        transcript = re.sub(r'\[.*?\]', '', transcript)  # Remove [NOISE], [MUSIC], etc.
        transcript = re.sub(r'<.*?>', '', transcript)    # Remove <unk>, <pad>, etc.
        transcript = re.sub(r'\s+', ' ', transcript)     # Normalize whitespace

        # Remove very short or repetitive transcripts
        if len(transcript) < 5:
            return ""

        # Check for repetitive patterns (common ASR error)
        words = transcript.split()
        if len(words) > 1:
            # If more than 70% of words are the same, likely an error
            unique_words = set(words)
            if len(unique_words) / len(words) < 0.3:
                return ""

        return transcript

In [13]:
class EnhancedMultiModalDataset(Dataset):
    """Enhanced dataset with ASR-generated transcripts and linguistic features"""

    def __init__(self, data_samples, audio_processor, tokenizer,
                 max_text_length=512, audio_max_length=16*16000,
                 image_size=(224, 224)):
        self.data_samples = data_samples
        self.audio_processor = audio_processor
        self.tokenizer = tokenizer
        self.max_text_length = max_text_length
        self.audio_max_length = audio_max_length
        self.image_size = image_size

        # Precompute linguistic features for efficiency
        self._precompute_linguistic_features()

    def _precompute_linguistic_features(self):
        """Precompute linguistic features that might be important for AD detection"""
        print("Precomputing linguistic features...")

        for sample in tqdm(self.data_samples, desc="Computing linguistic features"):
            transcript = sample['transcript']

            # Basic linguistic metrics
            words = transcript.split()
            sentences = re.split(r'[.!?]+', transcript)

            linguistic_features = {
                'word_count': len(words),
                'sentence_count': len([s for s in sentences if s.strip()]),
                'avg_word_length': np.mean([len(word) for word in words]) if words else 0,
                'unique_words': len(set(words)),
                'lexical_diversity': len(set(words)) / len(words) if words else 0,
                'pause_markers': transcript.count('[pause]') + transcript.count('...'),
                'filler_words': sum(1 for word in words if word.lower() in ['um', 'uh', 'er', 'ah']),
                'transcript_length': len(transcript)
            }

            sample['linguistic_features'] = linguistic_features

    def __len__(self):
        return len(self.data_samples)

    def __getitem__(self, idx):
        sample = self.data_samples[idx]

        # Process text
        text = sample['transcript']
        if not text or text.strip() == "":
            text = "No speech content detected in audio sample"

        try:
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_text_length,
                return_tensors='pt'
            )
        except Exception as e:
            print(f"Error tokenizing text for {sample['participant_id']}: {e}")
            # Create dummy encoding
            encoding = {
                'input_ids': torch.zeros(self.max_text_length, dtype=torch.long),
                'attention_mask': torch.zeros(self.max_text_length, dtype=torch.long),
                'token_type_ids': torch.zeros(self.max_text_length, dtype=torch.long)
            }

        # Process audio
        try:
            audio = self.audio_processor.load_audio(
                sample['audio_path'],
                max_length=self.audio_max_length
            )

            # Extract spectrogram features
            spectrogram = self.audio_processor.extract_mel_spectrogram(audio)

            # Resize for ViT input
            audio_features = self.audio_processor.resize_spectrogram_to_image(
                spectrogram, self.image_size
            )

        except Exception as e:
            print(f"Error processing audio for {sample['participant_id']}: {e}")
            # Create dummy audio features
            audio_features = np.random.rand(3, self.image_size[0], self.image_size[1])

        # Linguistic features
        ling_features = sample.get('linguistic_features', {})
        linguistic_vector = np.array([
            ling_features.get('word_count', 0),
            ling_features.get('sentence_count', 0),
            ling_features.get('avg_word_length', 0),
            ling_features.get('unique_words', 0),
            ling_features.get('lexical_diversity', 0),
            ling_features.get('pause_markers', 0),
            ling_features.get('filler_words', 0),
            ling_features.get('transcript_length', 0)
        ], dtype=np.float32)

        return {
            'input_ids': encoding['input_ids'].squeeze() if hasattr(encoding['input_ids'], 'squeeze') else encoding['input_ids'],
            'attention_mask': encoding['attention_mask'].squeeze() if hasattr(encoding['attention_mask'], 'squeeze') else encoding['attention_mask'],
            'audio_features': torch.FloatTensor(audio_features),
            'linguistic_features': torch.FloatTensor(linguistic_vector),
            'label': torch.LongTensor([sample['label']]).squeeze(),
            'participant_id': sample['participant_id'],
            'class_name': sample['class_name'],
            'transcript_preview': text[:100] + "..." if len(text) > 100 else text
        }

In [14]:
class SimpleAudioProcessor:
    def __init__(self, sample_rate=16000, n_mels=128):
        self.sample_rate = sample_rate
        self.n_mels = n_mels

    def load_audio(self, audio_path, max_length=None):
        try:
            audio, sr = librosa.load(audio_path, sr=self.sample_rate)
            if audio.ndim > 1:
                audio = np.mean(audio, axis=1)
            audio, _ = librosa.effects.trim(audio, top_db=20)
            if np.max(np.abs(audio)) > 0:
                audio = librosa.util.normalize(audio)

            if max_length is not None:
                if len(audio) > max_length:
                    start = (len(audio) - max_length) // 2
                    audio = audio[start:start + max_length]
                elif len(audio) < max_length:
                    pad_length = max_length - len(audio)
                    audio = np.pad(audio, (0, pad_length), mode='constant')

            return audio
        except:
            length = max_length if max_length else self.sample_rate * 10
            return np.random.randn(length) * 0.01

    def extract_mel_spectrogram(self, audio):
        try:
            mel_spec = librosa.feature.melspectrogram(
                y=audio, sr=self.sample_rate, n_mels=self.n_mels
            )
            log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            delta = librosa.feature.delta(log_mel_spec)
            delta2 = librosa.feature.delta(log_mel_spec, order=2)
            return np.stack([log_mel_spec, delta, delta2], axis=0)
        except:
            return np.random.randn(3, self.n_mels, 100)

    def resize_spectrogram_to_image(self, spectrogram, target_size=(224, 224)):
        try:
            if spectrogram.ndim == 3:
                resized_channels = []
                for i in range(spectrogram.shape[0]):
                    channel = spectrogram[i]
                    zoom_factors = [target_size[j] / channel.shape[j] for j in range(2)]
                    resized_channel = ndimage.zoom(channel, zoom_factors, order=1)
                    resized_channels.append(resized_channel)
                resized = np.stack(resized_channels, axis=0)
            else:
                zoom_factors = [target_size[i] / spectrogram.shape[i] for i in range(2)]
                resized = ndimage.zoom(spectrogram, zoom_factors, order=1)
                resized = np.stack([resized] * 3, axis=0)

            resized = (resized - resized.min()) / (resized.max() - resized.min() + 1e-8)
            return resized
        except:
            return np.random.rand(3, target_size[0], target_size[1])

In [15]:
class EnhancedMultiModalADClassifier(nn.Module):
    """Enhanced multimodal classifier for Alzheimer's detection"""

    def __init__(self, text_hidden_size=768, audio_hidden_size=768,
                 linguistic_feature_size=8, fusion_hidden_size=512,
                 num_classes=2, dropout=0.3):
        super().__init__()

        # Text encoder (BERT)
        self.text_encoder = BertModel.from_pretrained('bert-base-uncased')

        # Audio encoder (ViT for spectrograms)
        self.audio_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224')

        # Linguistic features processor
        self.linguistic_processor = nn.Sequential(
            nn.Linear(linguistic_feature_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Attention mechanism for modality fusion
        self.attention = nn.MultiheadAttention(
            embed_dim=fusion_hidden_size,
            num_heads=8,
            dropout=dropout
        )

        # Feature projections to common dimensionality
        self.text_projection = nn.Linear(text_hidden_size, fusion_hidden_size)
        self.audio_projection = nn.Linear(audio_hidden_size, fusion_hidden_size)
        self.linguistic_projection = nn.Linear(32, fusion_hidden_size)

        # Fusion layers
        self.fusion_layers = nn.Sequential(
            nn.Linear(fusion_hidden_size * 3, fusion_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(fusion_hidden_size, fusion_hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(fusion_hidden_size // 2, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_classes)
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        """Initialize model weights"""
        for module in [self.linguistic_processor, self.fusion_layers, self.classifier]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.zeros_(layer.bias)

    def forward(self, input_ids, attention_mask, audio_features, linguistic_features):
        batch_size = input_ids.size(0)

        # Text encoding
        text_output = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_features = text_output.pooler_output  # [batch_size, 768]

        # Audio encoding
        audio_output = self.audio_encoder(pixel_values=audio_features)
        audio_features = audio_output.pooler_output  # [batch_size, 768]

        # Linguistic features processing
        linguistic_processed = self.linguistic_processor(linguistic_features)  # [batch_size, 32]

        # Project to common dimensionality
        text_projected = self.text_projection(text_features)        # [batch_size, 512]
        audio_projected = self.audio_projection(audio_features)     # [batch_size, 512]
        linguistic_projected = self.linguistic_projection(linguistic_processed)  # [batch_size, 512]

        # Prepare for attention mechanism
        # Convert to [seq_len, batch_size, embed_dim] for attention
        modality_features = torch.stack([
            text_projected,
            audio_projected,
            linguistic_projected
        ], dim=0)  # [3, batch_size, 512]

        # Apply self-attention across modalities
        attended_features, attention_weights = self.attention(
            modality_features, modality_features, modality_features
        )  # [3, batch_size, 512]

        # Convert back to [batch_size, features]
        attended_features = attended_features.transpose(0, 1)  # [batch_size, 3, 512]

        # Flatten for fusion
        fused_input = attended_features.reshape(batch_size, -1)  # [batch_size, 1536]

        # Fusion
        fused_features = self.fusion_layers(fused_input)  # [batch_size, 256]

        # Classification
        logits = self.classifier(fused_features)  # [batch_size, 2]

        return {
            'logits': logits,
            'text_features': text_features,
            'audio_features': audio_features,
            'linguistic_features': linguistic_processed,
            'attention_weights': attention_weights,
            'fused_features': fused_features
        }